In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import random
import torch
from random import choice
import math
import gym
import copy
import time
import shutil
import tensorflow as tf
import tensorflow.compat.v1 as tf
import time
import gym
import numpy as np
import matplotlib.pyplot as plt
import os

from matplotlib import animation, rc
import matplotlib.pyplot as plt
from matplotlib import colors
from IPython.display import HTML
from matplotlib.animation import FuncAnimation
import matplotlib as mpl

%matplotlib inline

In [2]:
path2 = 'C:\\Users\\raven\\MAP-Elites-NEAT'

In [3]:
os.chdir(path2)

## Create Graph files

In [4]:
def create_graphs(num_models,num_inp,num_out):
    #remove all files in Mixed_NEAT folder
    if(os.getcwd()+'\\Mixed_NEAT' in [x[0] for x in os.walk(os.getcwd())]):
        shutil.rmtree(os.getcwd()+'\\Mixed_NEAT\\')
        
    #create Mixed_NEAT file
    os.mkdir(os.getcwd()+'\\Mixed_NEAT\\')
    
    for i in range(num_models):
        #create number of files needed
        fh=open(os.getcwd()+"\\Mixed_NEAT\\model_"+str(i)+".edgelist", "x")
        fh.close()
        
    #remove all files in Mixed_MAP folder
    if(os.getcwd()+'\\Mixed_MAP' in [x[0] for x in os.walk(os.getcwd())]):
        shutil.rmtree(os.getcwd()+'\\Mixed_MAP\\')

    #create Mixed_MAP file
    os.mkdir(os.getcwd()+'\\Mixed_MAP\\')

    #create number of files needed
    for x in range(dis_x):
        for y in range(dis_y):
            fh=open(os.getcwd()+"\\Mixed_MAP\\model_"+str(x)+"_"+str(y)+".edgelist", "x")
            fh.close()
            fh=open(os.getcwd()+"\\Mixed_MAP\\model_"+str(x)+"_"+str(y)+".txt", "x")
            fh.write(str(0.0))
            fh.close()

## Read model from file

In [5]:
def read_graph_NEAT(model_num,num_inp,num_out):
    fh=open(os.getcwd()+"\\Mixed_NEAT\\model_"+str(model_num)+".edgelist", "rb")
    G = nx.read_edgelist(fh,nodetype=int,create_using=nx.DiGraph(),data=(('weight',float),('activation',int),('inn_num',int)))
    fh.close() 
    
    #add input and output nodes
    for i in range(num_inp+num_out):
        G.add_node(i)
    return G

In [6]:
def read_graph_MAP(x_val,y_val,num_inp,num_out):
    fh=open(os.getcwd()+"\\Mixed_MAP\\model_"+str(x_val)+"_"+str(y_val)+".edgelist", "rb")
    G = nx.read_edgelist(fh,nodetype=int,create_using=nx.DiGraph(),data=(('weight',float),('activation',int),('inn_num',int)))
    fh.close() 
    
    #add input and output nodes
    for i in range(num_inp+num_out):
        G.add_node(i)
    return G

## Save Model

In [7]:
def write_graph_NEAT(model,model_num):
    nx.write_edgelist(model, os.getcwd()+"\\Mixed_NEAT\\model_"+str(model_num)+".edgelist",data=['weight','activation','inn_num'])

In [8]:
def write_graph_MAP(model,x_val,y_val,reward):
    nx.write_edgelist(model, os.getcwd()+"\\Mixed_MAP\\model_"+str(x_val)+"_"+str(y_val)+".edgelist",data=['weight','activation','inn_num'])
    fh=open(os.getcwd()+"\\Mixed_MAP\\model_"+str(x_val)+"_"+str(y_val)+".txt", "w")
    fh.write(str(reward))
    fh.close() 

## Mutate

In [9]:
def mutate(gene_model,num_models,max_inn_num,model_num):
    new_con_prob = .3
    con_to_node_prob = .3
    change_weights = .8
    
    if(random.random() < new_con_prob):
        add_con(gene_model,num_models,max_inn_num,model_num)
        
    if(random.random() < con_to_node_prob):
        con_to_node(gene_model,num_models,max_inn_num,model_num)
    
    if(random.random() < change_weights):
        new_weight(gene_model,model_num)    

## Add Connection

In [10]:
def add_con(model,num_models,max_inn_num,model_num):
    node_list = list(model.nodes)
    
    #get two random nodes
    length = len(node_list)
    node1 = node_list[random.randint(0,length-1)]
    node2 = node_list[random.randint(0,length-1)]
    
    #check if node2 is input node, node1 is an output node, or nodes are the same
    while( (node2<num_inp) or (node1>=num_inp and node1<num_inp+num_out) or (node1 == node2)):
        node1 = node_list[random.randint(0,length-1)]
        node2 = node_list[random.randint(0,length-1)]

    #check if adding edge will cause loop
    if(check_edges(model,node1,node2)):
        model.add_edge(node1,node2,weight=random.random()*3,activation=1,inn_num=inn_num((node1,node2),num_models,max_inn_num))
        max_inn_num[0]+=1

## Connection to Node

In [11]:
def con_to_node(model,num_models,max_inn_num,model_num):
    #if no connections, dont run rest of function
    temp = list(model.edges())
    if(len(temp)==0):
        return 0
    
    node1,node2 = temp[random.randint(0,len(temp)-1)]

    new_node = max(list(model.nodes))+1

    w1 = 1
    w2 = nx.get_edge_attributes(model,'weight').get((node1,node2))

    model.add_edge(node1,new_node,weight=w1,activation=1)
    model.add_edge(new_node,node2,weight=w2,activation=1)
    model.edges[node1,node2]['activation'] = 0
    
    model.edges[node1,new_node]['inn_num'] = inn_num((node1,new_node),num_models,max_inn_num)
    model.edges[new_node,node2]['inn_num'] = inn_num((new_node,node2),num_models,max_inn_num)

## Change weight

In [12]:
def new_weight(model,model_num):
    for edge in model.edges():
        if(random.random()<.1):
            #change weight
            weight = nx.get_edge_attributes(model,'weight').get(edge)
            model.edges[edge[0], edge[1]]['weight'] = random.random()*3
        else:
            #perturb weight
            weight = nx.get_edge_attributes(model,'weight').get(edge)
            model.edges[edge[0], edge[1]]['weight'] = model.edges[edge[0], edge[1]]['weight'] + random.random()*.3            

## Innovation numbers

In [13]:
'''def inn_num(edge,num_models,max_inn_num):
    #check if edge exists, if it does, set edge innovation number to match
    for model_num in range(num_models):
        try:
            model = read_graph(model_num,num_inp,num_out) 
        except:
            continue
        temp_dict = nx.get_edge_attributes(model,'inn_num')
        if edge in temp_dict.keys():
            return temp_dict[edge]
    
    #if new edge, then return current unused max innovation number
    max_inn_num[0]+=1
    return max_inn_num[0]-1'''

"def inn_num(edge,num_models,max_inn_num):\n    #check if edge exists, if it does, set edge innovation number to match\n    for model_num in range(num_models):\n        try:\n            model = read_graph(model_num,num_inp,num_out) \n        except:\n            continue\n        temp_dict = nx.get_edge_attributes(model,'inn_num')\n        if edge in temp_dict.keys():\n            return temp_dict[edge]\n    \n    #if new edge, then return current unused max innovation number\n    max_inn_num[0]+=1\n    return max_inn_num[0]-1"

In [14]:
def inn_num(edge,num_models,max_inn_num):
    #check if edge exists, if it does, set edge innovation number to match
    
    #check NEAT innovation numbers
    for model_num in range(num_models):
        try:
            NEAT_gene_model = read_graph_NEAT(model_num,num_inp,num_out) 
        except:
            continue
        temp_dict = nx.get_edge_attributes(NEAT_gene_model,'inn_num')
        if edge in temp_dict.keys():
            return temp_dict[edge]
        
    #check MAP innovation numbers
    for i in range(dis_x):
        for j in range(dis_y):
            MAP_gene_model = read_graph_MAP(i,j,num_inp,num_out)
            temp_dict = nx.get_edge_attributes(MAP_gene_model,'inn_num')
            if edge in temp_dict.keys():
                return temp_dict[edge]
    
    #if new edge, then return current unused max innovation number
    max_inn_num[0]+=1
    return max_inn_num[0]-1

In [15]:
'''def inn_num(edge,num_models,max_inn_num):
    #check if edge exists, if it does, set edge innovation number to match
    
    #check NEAT innovation numbers
    for model_num in range(num_models):
        NEAT_gene_model = read_graph_NEAT(model_num,num_inp,num_out) 
        temp_dict = nx.get_edge_attributes(NEAT_gene_model,'inn_num')
        if edge in temp_dict.keys():
            return temp_dict[edge]
        
    #check MAP innovation numbers
    for i in range(dis_x):
        for j in range(dis_y):
            MAP_gene_model = read_graph_MAP(i,j,num_inp,num_out)
            temp_dict = nx.get_edge_attributes(MAP_gene_model,'inn_num')
            if edge in temp_dict.keys():
                return temp_dict[edge]
    
    #if new edge, then return current unused max innovation number
    max_inn_num[0]+=1
    return max_inn_num[0]-1'''

"def inn_num(edge,num_models,max_inn_num):\n    #check if edge exists, if it does, set edge innovation number to match\n    \n    #check NEAT innovation numbers\n    for model_num in range(num_models):\n        NEAT_gene_model = read_graph_NEAT(model_num,num_inp,num_out) \n        temp_dict = nx.get_edge_attributes(NEAT_gene_model,'inn_num')\n        if edge in temp_dict.keys():\n            return temp_dict[edge]\n        \n    #check MAP innovation numbers\n    for i in range(dis_x):\n        for j in range(dis_y):\n            MAP_gene_model = read_graph_MAP(i,j,num_inp,num_out)\n            temp_dict = nx.get_edge_attributes(MAP_gene_model,'inn_num')\n            if edge in temp_dict.keys():\n                return temp_dict[edge]\n    \n    #if new edge, then return current unused max innovation number\n    max_inn_num[0]+=1\n    return max_inn_num[0]-1"

## Create TensorFlow graph

In [16]:
def get_tf_model(model,num_inp,num_out):
    #Create calc_dict of input nodes
    calc_dict = {}
    for i in range(num_inp): calc_dict[i]=tf.placeholder(tf.float32,[1])
        
    #get list of uncalculated nodes            
    temp_uncalc_node_list = []
    for i in model.nodes():
        if(i not in calc_dict):
            temp_uncalc_node_list.append(i)

    #calculate every node in uncalclated node list until list is empty
    index = -1
    while(len(temp_uncalc_node_list)>0):
        index+=1
        node = temp_uncalc_node_list[index]
        
        #if any input node is uncalculated, break, else, calculate current node
        temp_list = list(model.in_edges(node))
        temp_bool = False
        for i in temp_list:
            if i[0] not in calc_dict:
                temp_bool = True
                break
        if(temp_bool):
            continue

        #sum over all edge weight multiplied by the child nodes output
        tensor_sum = tf.zeros([1],dtype=tf.float32)
        for cur_edge in temp_list:
            weight = nx.get_edge_attributes(model,'weight').get(cur_edge)
            weight = tf.constant(weight,dtype=tf.float32)

            activation = nx.get_edge_attributes(model,'activation').get(cur_edge)
            activation = tf.constant(activation,dtype=tf.float32)

            node_output = calc_dict[cur_edge[0]]
            tensor_sum = node_output*weight*activation + tensor_sum

        #add calculated node to calc_dict and remove node from uncalc_node list
        calc_dict[node] = tf.math.tanh(tensor_sum)
        temp_uncalc_node_list.pop(index)
        index = -1
                
    #return list of output nodes
    out_list = []
    for i in range(num_inp,num_inp+num_out):
        if(i not in calc_dict):
            #not all output nodes are accounted for and are missing in calc_dict
            print("!!!!!!!!!!!!!!ERROR!!!!!!!!!!!!!!!!!")
        else:
            out_list.append(calc_dict[i])

    return out_list,calc_dict

## Speciate

In [17]:
#problem with c2 here
def speciate(dict_rew_list,num_models,c1,c2,c3):
    threshold = 3.45
    max_species = 0
    
    #double for loop, 
    #create list of model_numbers with the same species
    #[ [0,1,2,3], [4], [5,6,7,8], [9]]   Here, there are 4 species with 10 networks
    used_model_num_list = [i for i in range(num_models)]
    spec_list = [[(dict_rew_list[0],0)]] #initialized with the first model in its own species
    max_species = 0
    for model_num2 in used_model_num_list:
        found_species = False
        for species in spec_list:
            model_num1 = species[0][1]
            #if model number are the same, skip
            if(model_num1 == model_num2):
                found_species = True
                break
            
            #read the two models and calculate the difference
            gene_model1 = read_graph_NEAT(model_num1,num_inp,num_out)
            gene_model2 = read_graph_NEAT(model_num2,num_inp,num_out)
            diff = model_dif(gene_model1,gene_model2,c1,c2,c3)
            
            #if difference is below threshold, set model1 species to model2 species
            if(diff < threshold):
                #find graph in species lists
                species.append((dict_rew_list[model_num2],model_num2))
                found_species = True
                break
        if(found_species==False):
            spec_list.append([(dict_rew_list[model_num2],model_num2)])
            
    return spec_list

## Speciate helper function (Adjusted fitness)

In [18]:
def model_dif(gene_model1,gene_model2,c1,c2,c3):
    delta_sum = 0
    num_disjoint = 1
    num_excess = 1
    num_weights = 1
    weight_sum = 0
    
    temp_dict1 = nx.get_edge_attributes(gene_model1,'weight')
    temp_dict2 = nx.get_edge_attributes(gene_model2,'weight')
    for edge in temp_dict1:
        if(edge in temp_dict2):
            #weight difference
            num_weights+=1
            weight1 = nx.get_edge_attributes(gene_model1,'weight').get(edge)
            weight2 = nx.get_edge_attributes(gene_model2,'weight').get(edge)
            weight_sum += abs(weight2-weight1)
        else:
            # disjoint
            num_disjoint+=1
    
    for edge in temp_dict2:
        #excess
        if(edge not in temp_dict1):
            num_disjoint+=1
        
    delta_sum = (c1*num_excess)/(num_excess + num_disjoint) + (c2*num_disjoint)/(num_excess + num_disjoint) + (c3*weight_sum)/num_weights
    return delta_sum

## Crossover

In [19]:
'''def crossover(reward_list,num_models,max_inn_num,keep_amt,c1,c2,c3,MAP_to_NEAT_per):
    #need to do speciation before deletion
    temp_reward_list = [(sub[1], sub[0]) for sub in reward_list] 
    dict_rew_list = dict(temp_reward_list)
    
    #speciate
    spec_list = speciate(dict_rew_list,num_models,c1,c2,c3)
    print(len(spec_list))
    
    #adjust fitness
    #get average reward of population
    avg_rew = 0
    for i in reward_list:
        avg_rew += i[0]
    avg_rew = avg_rew
    
    #get number of graphs per species
    num_children_list = []
    for spec in spec_list:
        per_children = 0
        for model_rew,model_num in spec:
            per_children += model_rew
            
        per_children = per_children/avg_rew
        num_children_list.append(per_children)
  
    #call delete graphs
    del_amt = int((1-keep_amt)*num_models)
    unused_model_num_list,used_model_num_list,del_rew_list = del_graphs(temp_reward_list,num_models,del_amt,spec_list,num_children_list)   
    
    unused_model_num_list.sort()
    used_model_num_list.sort()

    
    #add MAP graphs to NEAT
    prob_crossover_MAP = .5
    MAP_to_NEAT_amt = int(MAP_to_NEAT_per*num_models)
    index_array = get_existing_graphs(MAP_to_NEAT_amt,dis_x,dis_y,num_inp,num_out)
    for x_val,y_val in index_array:
        if(len(unused_model_num_list)<1):
            break
        MAP_gene_model = read_graph_MAP(x_val,y_val,num_inp,num_out)
        #mutate(MAP_gene_model,num_models,max_inn_num,unused_model_num_list[0])
        #crossover MAP graph
        #if(random.random()<prob_crossover_MAP):
        #    #This crossovers MAP to MAP graphs. This is not needed if they are mutatated and added to NEAT graphs
        #    ind = random.randint(0,MAP_to_NEAT_amt-1)
        #    x,y = index_array[ind]
        #    gene_model1 = read_graph_MAP(x,y,num_inp,num_out)
        #    MAP_gene_model = get_child_gene_model(MAP_gene_model,gene_model1)
        write_graph_NEAT(MAP_gene_model,unused_model_num_list[0])
        added_num = unused_model_num_list.pop(0) 
        used_model_num_list.append(added_num)
       
    
    
    #crossover
    prob_mutate = .8
    new_num_models = len(used_model_num_list)
    while(len(unused_model_num_list)>0):
        #mutate graph
        if(random.random()<prob_mutate or len(unused_model_num_list)==1):
            rand_ind = random.randint(0,len(used_model_num_list)-1)
            model_num = used_model_num_list[rand_ind]
            gene_model = read_graph_NEAT(model_num,num_inp,num_out)   
            temp_gene_model = gene_model.copy()
            mutate(temp_gene_model,num_models,max_inn_num,unused_model_num_list[0])
            write_graph_NEAT(temp_gene_model,unused_model_num_list[0])
            added_num = unused_model_num_list.pop(0)  
        #crossover graph
        else:
            #choose two random and different indeces
            rand_ind1 = random.randint(0,len(used_model_num_list)-1)
            rand_ind2 = choice([i for i in range(0,len(used_model_num_list)-1) if i not in [rand_ind1]])

            #get model numbers
            model_num1 = used_model_num_list[rand_ind1]
            model_num2 = used_model_num_list[rand_ind2]

            #get models from model numbers        
            gene_model1 = read_graph_NEAT(model_num1,num_inp,num_out)        
            gene_model2 = read_graph_NEAT(model_num2,num_inp,num_out)

            #get new child model, write model, and pop used model number from list
            child_gene_model = get_child_gene_model(gene_model1,gene_model2)
            #need to get innovation number of child gene
            write_graph_NEAT(child_gene_model,unused_model_num_list[0])
            added_num = unused_model_num_list.pop(0)  
'''

'def crossover(reward_list,num_models,max_inn_num,keep_amt,c1,c2,c3,MAP_to_NEAT_per):\n    #need to do speciation before deletion\n    temp_reward_list = [(sub[1], sub[0]) for sub in reward_list] \n    dict_rew_list = dict(temp_reward_list)\n    \n    #speciate\n    spec_list = speciate(dict_rew_list,num_models,c1,c2,c3)\n    print(len(spec_list))\n    \n    #adjust fitness\n    #get average reward of population\n    avg_rew = 0\n    for i in reward_list:\n        avg_rew += i[0]\n    avg_rew = avg_rew\n    \n    #get number of graphs per species\n    num_children_list = []\n    for spec in spec_list:\n        per_children = 0\n        for model_rew,model_num in spec:\n            per_children += model_rew\n            \n        per_children = per_children/avg_rew\n        num_children_list.append(per_children)\n  \n    #call delete graphs\n    del_amt = int((1-keep_amt)*num_models)\n    unused_model_num_list,used_model_num_list,del_rew_list = del_graphs(temp_reward_list,num_models,del

In [20]:
def crossover(reward_list,num_models,max_inn_num,keep_amt,c1,c2,c3,MAP_to_NEAT_per):
    #need to do speciation before deletion
    temp_reward_list = [(sub[1], sub[0]) for sub in reward_list] 
    dict_rew_list = dict(temp_reward_list)
    
    #speciate
    spec_list = speciate(dict_rew_list,num_models,c1,c2,c3)
    print(len(spec_list))
    
    #get average reward of population
    avg_rew = 0
    for i in reward_list:
        avg_rew += i[0]
    avg_rew = avg_rew
    
    #get number of graphs per species & adjust fitness
    num_children_list = []
    for spec in spec_list:
        per_children = 0
        for model_rew,model_num in spec:
            per_children += model_rew
            
        per_children = per_children/avg_rew
        num_children_list.append(per_children)
  
    #call delete graphs
    del_amt = int((1-keep_amt)*num_models)
    unused_model_num_list,used_model_num_list,del_rew_list = del_graphs(temp_reward_list,num_models,del_amt,spec_list,num_children_list)   
    
    unused_model_num_list.sort()
    used_model_num_list.sort()
    
    #add MAP graphs to NEAT
    prob_crossover_MAP = .5
    MAP_to_NEAT_amt = int(MAP_to_NEAT_per*num_models)
    index_array = get_existing_graphs(MAP_to_NEAT_amt,dis_x,dis_y,num_inp,num_out)
    for x_val,y_val in index_array:
        if(len(unused_model_num_list)<1):
            break
        MAP_gene_model = read_graph_MAP(x_val,y_val,num_inp,num_out)
        mutate(MAP_gene_model,num_models,max_inn_num,unused_model_num_list[0])
        write_graph_NEAT(MAP_gene_model,unused_model_num_list[0])
        added_num = unused_model_num_list.pop(0) 
        used_model_num_list.append(added_num)
    
    #crossover
    prob_mutate = .8
    new_num_models = len(used_model_num_list)
    while(len(unused_model_num_list)>0):
        #choose two random and different indeces
        rand_ind1 = random.randint(0,len(used_model_num_list)-1)
        rand_ind2 = choice([i for i in range(0,len(used_model_num_list)-1) if i not in [rand_ind1]])

        #get model numbers
        model_num1 = used_model_num_list[rand_ind1]
        model_num2 = used_model_num_list[rand_ind2]

        #get models from model numbers        
        gene_model1 = read_graph_NEAT(model_num1,num_inp,num_out)        
        gene_model2 = read_graph_NEAT(model_num2,num_inp,num_out)

        #get new child model, write model, and pop used model number from list
        child_gene_model = get_child_gene_model(gene_model1,gene_model2)
        #need to get innovation number of child gene
        write_graph_NEAT(child_gene_model,unused_model_num_list[0])
        added_num = unused_model_num_list.pop(0)  

## Crossover helper function (Delete models)

In [21]:
def del_graphs(reward_list,num_models,del_amt,spec_list,num_children_list):
    unused_model_num_list = []
    used_model_num_list = []
    del_rew_list = []
    path = os.getcwd()+"\\Mixed_NEAT\\"
    
    spec_count = -1
    for perc in num_children_list:
        #get number of models to delete per species
        spec_count+=1
        spec_del_amt = int(perc*del_amt)
        spec_rew_list = spec_list[spec_count]
        spec_rew_list.sort()
        
        #delete graph and add model number to unused list, or leave graph and add model number to used list
        model_count = -1
        for i in spec_list[spec_count]:
            model_count+=1
            if(model_count < spec_del_amt):
                os.remove(path+"model_"+str(spec_rew_list[model_count][1])+".edgelist")     
                unused_model_num_list.append(spec_rew_list[model_count][1])
            else:
                used_model_num_list.append(spec_rew_list[model_count][1])
                del_rew_list.append(spec_rew_list[model_count])

    return unused_model_num_list,used_model_num_list,del_rew_list

## Get child model given parents

In [22]:
def get_child_gene_model(gene_model1,gene_model2):
    act_prob = .7
    child_model = nx.DiGraph()
    
    #get attributes of each parent graph
    weight_dict1 = nx.get_edge_attributes(gene_model1,'weight')
    weight_dict2 = nx.get_edge_attributes(gene_model2,'weight')
    act_dict1 = nx.get_edge_attributes(gene_model1,'activation')
    act_dict2 = nx.get_edge_attributes(gene_model2,'activation')
    inn_dict1 = nx.get_edge_attributes(gene_model1,'inn_num')
    inn_dict2 = nx.get_edge_attributes(gene_model2,'inn_num')
    
    #loop over first parent to see if nodes 
    for edge in weight_dict1:
        #if edge is in both models, 50% chance to choose either
        weight = 0
        activation = 1
        innovation = 0
        if(edge in weight_dict2):
            if(random.random() < .5):
                weight = weight_dict1.get(edge)
                activation = act_dict1.get(edge)
                innovation = inn_dict1.get(edge)
            else:
                weight = weight_dict2.get(edge)
                activation = act_dict2.get(edge)
                innovation = inn_dict2.get(edge)
        else:
            weight = weight_dict1.get(edge)
            activation = act_dict1.get(edge)
            innovation = inn_dict1.get(edge)
            
        #set activation to 1 with probability
        if(activation == 0):
            if(random.random()<act_prob):
                activation = 1
                
        #check if edge can be added without creating a circuit
        if(check_edges(child_model,edge[0],edge[1])):
            child_model.add_edge(edge[0],edge[1],weight=weight,activation=activation,inn_num=innovation)
        
    for edge in weight_dict2:
        #if edge is in both models, 50% chance to choose either
        if(edge not in weight_dict1):
            weight = 0
            activation = 1
            weight = weight_dict2.get(edge)
            activation = act_dict2.get(edge)
            innovation = inn_dict2.get(edge)
            
        #set activation to 1 with probability
        if(activation == 0):
            if(random.random()<act_prob):
                activation = 1
            
        #check if edge can be added without creating a circuit
        if(check_edges(child_model,edge[0],edge[1])):
            child_model.add_edge(edge[0],edge[1],weight=weight,activation=activation,inn_num=innovation)  
    return child_model

## Check for loops in graph

In [23]:
def check_edges(model,node1,node2):
    if((node2,node1) in model.edges() or (node1,node2) in model.edges()):
        return False
            
    temp_model = model.copy()
    temp_model.add_edge(node1,node2,weight=random.randrange(-3,3))
    
    try:
        #if model has cycles, switch node1 and node2
        nx.find_cycle(temp_model, orientation='original')
    except:
        #model.add_edge(node1,node2,weight=random.random(),activation=1,inn_num=inn_num((node1,node2),num_models,max_inn_num))
        return True
    return False

## Get existing graphs from MAP-Elites

In [24]:
def get_existing_graphs(num_models,dis_x,dis_y,num_inp,num_out):
    
    #get list of ones and zeros if graph exists
    available_graphs = []
    for i in range(dis_x):
        for j in range(dis_y):
            MAP_gene_model = read_graph_MAP(i,j,num_inp,num_out)
            if(not nx.is_empty(MAP_gene_model)):
                available_graphs.append((i,j))
         
    #if empty, add index (0,0)
    if(len(available_graphs)==0):
        available_graphs.append((0,0))
    index_array = random.choices(available_graphs, k=num_models)
    return index_array

In [25]:
'''def get_existing_graphs(num_models,dis_x,dis_y,num_inp,num_out):
    
    #what if graphs got pulled from map elites based on howw well they did?
    
    #get list of ones and zeros if graph exists
    available_graphs = []
    available_graph_rewards = []
    for i in range(dis_x):
        for j in range(dis_y):
            MAP_gene_model = read_graph_MAP(i,j,num_inp,num_out)
            if(not nx.is_empty(MAP_gene_model)):
                available_graphs.append((i,j))
                
                #get current graphs reward
                fh=open(os.getcwd()+"\\Mixed_MAP\\model_"+str(i)+"_"+str(j)+".txt", "r")
                reward = float(fh.read())
                fh.close()
                available_graph_rewards.append(reward)
    
    #if empty, add index (0,0)
    if(len(available_graphs)==0):
        available_graphs.append((0,0))
        available_graph_rewards.append(1.0)
        
    norm_avail_graph_r = [i/np.sum(np.array(available_graph_rewards)) for i in np.array(available_graph_rewards)]
    index_array = random.choices(available_graphs, weights=norm_avail_graph_r,k=num_models)
    return index_array'''

'def get_existing_graphs(num_models,dis_x,dis_y,num_inp,num_out):\n    \n    #what if graphs got pulled from map elites based on howw well they did?\n    \n    #get list of ones and zeros if graph exists\n    available_graphs = []\n    available_graph_rewards = []\n    for i in range(dis_x):\n        for j in range(dis_y):\n            MAP_gene_model = read_graph_MAP(i,j,num_inp,num_out)\n            if(not nx.is_empty(MAP_gene_model)):\n                available_graphs.append((i,j))\n                \n                #get current graphs reward\n                fh=open(os.getcwd()+"\\Mixed_MAP\\model_"+str(i)+"_"+str(j)+".txt", "r")\n                reward = float(fh.read())\n                fh.close()\n                available_graph_rewards.append(reward)\n    \n    #if empty, add index (0,0)\n    if(len(available_graphs)==0):\n        available_graphs.append((0,0))\n        available_graph_rewards.append(1.0)\n        \n    norm_avail_graph_r = [i/np.sum(np.array(available_graph_rew

## Hyper-parameters

In [26]:
num_inp = 24
num_out = 4
num_bias = 10
num_inp = num_inp + num_bias

#changing num_models limits amount of saved models: fix this
num_models = 200
trials = 1
num_game_steps = 500
max_inn_num = [0]

generations = 1000
keep_amt = .3
MAP_to_NEAT_per = .3

c1 = 3.0
c2 = 3.0
c3 = 3.0

dis_x = 100
dis_y = 100#num_game_steps

## Run networks

In [ ]:
tf.reset_default_graph()
env = gym.make('BipedalWalker-v3')
create_graphs(num_models,num_inp,num_out)

full_rew_list = []
high_rew_list = []
with tf.Session() as sess:
    for gen in range(generations):
        total_a = time.time()

        #empty reward
        reward_list = []
        
        #run over all models
        for model_num in range(num_models):
            #get model from file
            gene_model = read_graph_NEAT(model_num,num_inp,num_out)
            
            #Mutation
            mutate(gene_model,num_models,max_inn_num,model_num)
            
            print("model num: {} and {}".format(model_num,gene_model.edges()))
            full_sum = 0
            if(nx.is_empty(gene_model)):
                reward_list.append((-38250.0,model_num))
                continue
                
            game_step_sum = -1
            g = tf.Graph()
            with g.as_default():
                with tf.Session() as sess:
                    #turn gene model into tensorflow model
                    tf_outputs,calc_dict = get_tf_model(gene_model,num_inp,num_out)

                    #heiristic initializations
                    avg_horizontal_speed = 0
                    perc_legs_on_ground = 0

                    for trial in range(trials):
                        #get inputs
                        observation = env.reset()
                        observation = np.append(observation,[1])

                        #run model
                        a = time.time()
                        model_r_sum = 0
                        for game_steps in range(num_game_steps):
                            game_step_sum+=1
                            #set up feed dictionary for model input
                            feed_dict = {}
                            for i in range(len(observation)): feed_dict[calc_dict[i]] = [observation[i]]
                            for i in range(len(observation),num_inp): feed_dict[calc_dict[i]] = [1.0]
                            
                            actions = sess.run(tf_outputs,feed_dict=feed_dict)  #this is what costs the most time
                            action = np.array([actions[i][0] for i in range(num_out)])

                            #take action
                            observation, reward, done, info = env.step(action)

                            #add observation to heiristics
                            avg_horizontal_speed += observation[2]
                            perc_legs_on_ground += observation[-11]

                            #append reward
                            full_sum+=reward
                            if(done):
                                env.close()
                                break
                        b = time.time()
                        print("run model: {}".format(b-a))
                    full_sum = avg_horizontal_speed*game_step_sum
                    if(game_step_sum!=num_game_steps-1):
                        full_sum-=40000
                    reward_list.append((full_sum/trials,model_num))

                    avg_horizontal_speed /= num_game_steps*trials
                    perc_legs_on_ground /= num_game_steps*trials

                    if(avg_horizontal_speed < 0):
                        avg_horizontal_speed = 0

                    x_val = int(avg_horizontal_speed*dis_x*5)
                    y_val = int(perc_legs_on_ground*dis_y)

                    if(y_val >= dis_y):
                        y_val = dis_y-1

                    if(x_val >= dis_x):
                        x_val = dis_x-1
                    
                    #if current reward > old_reward, swap saved model
                    fh=open(os.getcwd()+"\\Mixed_MAP\\model_"+str(x_val)+"_"+str(y_val)+".txt", "r")
                    old_reward = float(fh.read())
                    fh.close()

                    MAP_gene_model = read_graph_MAP(x_val,y_val,num_inp,num_out)
                    if(full_sum > old_reward or nx.is_empty(MAP_gene_model)):
                        write_graph_MAP(gene_model,x_val,y_val,full_sum)
                        print("{} > {} at {},{} and {} and {}".format(full_sum,old_reward,x_val,y_val,gene_model.edges(),MAP_gene_model.edges()))

        #Crossover
        crossover(reward_list,num_models,max_inn_num,keep_amt,c1,c2,c3,MAP_to_NEAT_per)

        #calculate average reward
        sum_rew = 0
        for i in reward_list:
            sum_rew+= i[0]
        avg_rew = sum_rew/num_models
        #reward_list.append(avg_rew)
        full_rew_list.append(avg_rew)
        high_rew_list.append(max(reward_list))
        total_b = time.time()
        print("Generation: {} Average reward: {} Time: {} Highest Reward: {}".format(gen,avg_rew,total_b-total_a,max(reward_list)))
sess.close()
total_b = time.time()

C:\Users\raven\anaconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


model num: 0 and []
model num: 1 and []
model num: 2 and []
model num: 3 and []
model num: 4 and []
model num: 5 and []
model num: 6 and [(19, 34)]
run model: 0.4431002140045166
-37699.16434609239 > 0.0 at 13,31 and [(19, 34)] and []
model num: 7 and []
model num: 8 and []
model num: 9 and [(27, 35)]
run model: 0.19504451751708984
-39061.782213664876 > 0.0 at 13,10 and [(27, 35)] and []
model num: 10 and []
model num: 11 and []
model num: 12 and []
model num: 13 and []
model num: 14 and []
model num: 15 and []
model num: 16 and []
model num: 17 and [(25, 35), (25, 38), (38, 35)]
run model: 0.18204092979431152
-39231.04599154554 > 0.0 at 11,9 and [(25, 35), (25, 38), (38, 35)] and []
model num: 18 and []
model num: 19 and []
model num: 20 and [(26, 37)]
run model: 0.3630814552307129
-41897.83941263631 > 0.0 at 0,21 and [(26, 37)] and []
model num: 21 and []
model num: 22 and []
model num: 23 and [(4, 35)]
run model: 0.35707998275756836
-38012.1457195282 > 0.0 at 15,14 and [(4, 35)] and 

model num: 168 and [(4, 35)]
run model: 0.3580808639526367
-38053.93989390395 > 0.0 at 15,11 and [(4, 35)] and []
model num: 169 and []
model num: 170 and []
model num: 171 and []
model num: 172 and []
model num: 173 and [(23, 35)]
run model: 0.19004297256469727
-39188.89877417132 > 0.0 at 12,10 and [(23, 35)] and []
model num: 174 and [(12, 37), (12, 38), (38, 37)]
run model: 0.2800638675689697
-38519.08198784411 > 0.0 at 14,11 and [(12, 37), (12, 38), (38, 37)] and []
model num: 175 and [(19, 35)]
run model: 0.19704437255859375
-39121.49312898218 > -39188.89877417132 at 12,10 and [(19, 35)] and [(23, 35)]
model num: 176 and [(15, 34)]
run model: 1.3373031616210938
5142.138739091145 > 0.0 at 10,88 and [(15, 34)] and []
model num: 177 and []
model num: 178 and []
model num: 179 and []
model num: 180 and []
model num: 181 and []
model num: 182 and [(26, 35)]
run model: 0.18804216384887695
-39220.131468706204 > -39231.04599154554 at 11,9 and [(26, 35)] and [(25, 35), (25, 38), (38, 35)]


run model: 1.3172972202301025
4919.866534009291 > 0.0 at 9,47 and [(28, 34)] and []
model num: 46 and [(12, 37), (12, 38), (38, 37), (14, 35)]
run model: 0.23505306243896484
-39189.65381102681 > 0.0 at 11,4 and [(12, 37), (12, 38), (38, 37), (14, 35)] and []
model num: 47 and [(30, 34), (23, 35)]
run model: 0.11502599716186523
-40090.72533191591 > -40090.82019634186 at 0,0 and [(30, 34), (23, 35)] and [(26, 35), (21, 34)]
model num: 48 and [(14, 35), (18, 36), (32, 35)]
run model: 1.320289134979248
516.8523796492768 > 0.0 at 1,94 and [(14, 35), (18, 36), (32, 35)] and []
model num: 49 and [(28, 34), (5, 35), (7, 37)]
run model: 0.13303017616271973
model num: 50 and [(18, 36), (18, 38), (38, 36)]
run model: 0.31307244300842285
model num: 51 and [(15, 34)]
run model: 1.27528977394104
4861.048272339485 > 0.0 at 9,87 and [(15, 34)] and []
model num: 52 and [(4, 35), (18, 37), (18, 38), (38, 37), (16, 37)]
run model: 0.41510629653930664
-42943.18642653704 > 0.0 at 0,24 and [(4, 35), (18, 37

run model: 0.24605607986450195
model num: 116 and [(23, 36), (23, 38), (38, 36), (38, 39), (39, 36)]
run model: 0.3520796298980713
-41245.75590776652 > -41250.900099515915 at 0,12 and [(23, 36), (23, 38), (38, 36), (38, 39), (39, 36)] and [(18, 36), (18, 38), (38, 36)]
model num: 117 and []
model num: 118 and [(23, 35), (23, 38), (38, 35), (10, 37), (30, 34)]
run model: 0.40909266471862793
-38044.357906230536 > 0.0 at 12,22 and [(23, 35), (23, 38), (38, 35), (10, 37), (30, 34)] and []
model num: 119 and []
model num: 120 and [(14, 35), (18, 36), (28, 34), (28, 38), (5, 35), (38, 34)]
run model: 0.12823987007141113
model num: 121 and [(13, 36), (13, 38), (13, 39), (38, 36), (39, 36)]
run model: 0.44547414779663086
-41656.935287347434 > -41870.8900735569 at 0,22 and [(13, 36), (13, 38), (13, 39), (38, 36), (39, 36)] and [(11, 37)]
model num: 122 and [(18, 36), (18, 38), (38, 36), (38, 39), (39, 36), (26, 35), (21, 34)]
run model: 0.27706241607666016
-38675.9572569409 > -39142.67076439231

...

model num: 141 and [(28, 34), (28, 38), (38, 34), (38, 39), (38, 41), (39, 34), (39, 36), (41, 34), (30, 34), (30, 38), (30, 39), (13, 34), (13, 38), (13, 39), (15, 34), (11, 36), (0, 37), (12, 34), (12, 38), (12, 39), (33, 34), (33, 40), (40, 34), (3, 34), (19, 34), (19, 38), (7, 34), (1, 36), (1, 39), (29, 39)]
run model: 1.3433029651641846
model num: 142 and [(19, 34), (19, 38), (19, 39), (38, 34), (39, 38), (0, 37), (12, 38), (12, 40), (40, 38)]
run model: 0.2830636501312256
model num: 143 and [(28, 34), (28, 38), (28, 41), (38, 34), (41, 34), (30, 34), (30, 38), (30, 39), (39, 34), (19, 34), (19, 38), (0, 37), (0, 34), (0, 40), (40, 34), (15, 34), (15, 42), (42, 34), (3, 34), (3, 37), (17, 39), (7, 34), (1, 36), (13, 34), (13, 38), (12, 38)]
run model: 1.3493051528930664
model num: 144 and [(15, 34), (15, 40), (40, 34), (10, 34), (10, 38), (38, 34), (2, 36), (9, 34), (9, 38), (28, 34), (28, 38), (28, 39), (28, 41), (39, 34), (41, 38), (20, 38), (7, 38), (3, 35)]
run model: 1.35930

run model: 0.11202573776245117
model num: 165 and [(28, 34), (28, 38), (38, 34), (30, 34), (30, 38), (30, 39), (39, 34), (39, 41), (19, 34), (19, 38), (0, 37), (0, 34), (0, 40), (40, 34), (15, 34), (3, 34), (3, 37), (17, 39), (7, 34), (1, 36), (13, 34), (13, 38), (12, 38), (5, 38), (33, 39), (41, 34)]
run model: 1.4578373432159424
19505.88684443681 > 0.0 at 39,39 and [(28, 34), (28, 38), (38, 34), (30, 34), (30, 38), (30, 39), (39, 34), (39, 41), (19, 34), (19, 38), (0, 37), (0, 34), (0, 40), (40, 34), (15, 34), (3, 34), (3, 37), (17, 39), (7, 34), (1, 36), (13, 34), (13, 38), (12, 38), (5, 38), (33, 39), (41, 34)] and []
model num: 166 and [(19, 34), (19, 38), (19, 39), (19, 43), (38, 34), (38, 42), (39, 38), (39, 34), (42, 34), (0, 37), (0, 34), (12, 38), (12, 41), (12, 36), (41, 38), (33, 38), (33, 40), (40, 38), (40, 34), (28, 34), (28, 38), (30, 34), (30, 38), (30, 40), (3, 36), (2, 36), (6, 38), (43, 34)]
run model: 1.6813807487487793
model num: 167 and [(30, 34), (12, 37), (12, 

run model: 1.409318447113037
model num: 190 and [(19, 34), (19, 38), (19, 39), (38, 34), (39, 38), (39, 34), (0, 37), (0, 34), (12, 38), (15, 34), (30, 34), (30, 38), (30, 39), (3, 34), (3, 37), (3, 36), (28, 34), (28, 38), (28, 40), (40, 38), (40, 41), (41, 38), (17, 39), (14, 39)]
run model: 1.3993151187896729
3406.1392346343614 > 0.0 at 6,66 and [(19, 34), (19, 38), (19, 39), (38, 34), (39, 38), (39, 34), (0, 37), (0, 34), (12, 38), (15, 34), (30, 34), (30, 38), (30, 39), (3, 34), (3, 37), (3, 36), (28, 34), (28, 38), (28, 40), (40, 38), (40, 41), (41, 38), (17, 39), (14, 39)] and []
model num: 191 and [(15, 34), (15, 38), (38, 34), (38, 39), (38, 40), (39, 34), (40, 34), (13, 34), (13, 38), (13, 39), (13, 41), (41, 39), (28, 34), (28, 38), (28, 39), (28, 40), (26, 34), (14, 34), (30, 34), (30, 38), (30, 39), (30, 41), (0, 37), (0, 34), (7, 34), (7, 38), (2, 36), (3, 34), (3, 38), (3, 37), (19, 34), (19, 38), (19, 39), (10, 34), (10, 38), (9, 34), (9, 38), (20, 38), (25, 34), (11, 3

run model: 1.3673079013824463
model num: 7 and [(28, 34), (28, 38), (28, 41), (28, 44), (38, 34), (41, 38), (44, 38), (30, 34), (30, 38), (30, 39), (30, 42), (30, 45), (39, 34), (42, 38), (45, 38), (19, 34), (19, 38), (19, 43), (43, 34), (0, 37), (0, 34), (0, 40), (40, 34), (15, 34), (3, 34), (3, 37), (17, 39), (7, 34), (1, 36), (13, 34), (13, 38), (26, 40), (31, 38)]
run model: 1.3703093528747559
13286.5211987641 > 0.0 at 26,29 and [(28, 34), (28, 38), (28, 41), (28, 44), (38, 34), (41, 38), (44, 38), (30, 34), (30, 38), (30, 39), (30, 42), (30, 45), (39, 34), (42, 38), (45, 38), (19, 34), (19, 38), (19, 43), (43, 34), (0, 37), (0, 34), (0, 40), (40, 34), (15, 34), (3, 34), (3, 37), (17, 39), (7, 34), (1, 36), (13, 34), (13, 38), (26, 40), (31, 38)] and []
model num: 8 and [(28, 34), (28, 38), (38, 34), (30, 34), (30, 38), (30, 39), (39, 34), (19, 34), (19, 38), (0, 37), (0, 34), (0, 40), (40, 34), (15, 34), (3, 34), (3, 37), (17, 39), (7, 34), (1, 36), (13, 34), (13, 38), (12, 38), (

run model: 1.418321132659912
model num: 28 and [(28, 34), (28, 38), (28, 40), (38, 34), (38, 39), (40, 38), (39, 34), (11, 36), (30, 34), (30, 38), (0, 37), (19, 34), (19, 38)]
run model: 1.3483049869537354
model num: 29 and [(28, 34), (28, 38), (28, 41), (38, 34), (38, 39), (41, 34), (39, 34), (30, 34), (30, 38), (30, 40), (30, 39), (40, 34), (0, 37), (0, 34), (19, 34), (19, 38), (19, 40), (15, 34), (15, 38), (26, 38)]
run model: 0.414093017578125
model num: 30 and [(13, 34), (13, 38), (13, 39), (38, 34), (38, 39), (39, 34), (28, 34), (28, 38), (28, 40), (40, 38), (15, 34), (26, 34), (26, 41), (14, 34), (30, 34), (30, 38), (30, 39), (30, 41), (41, 39), (0, 37), (0, 34), (19, 34), (19, 38), (3, 34), (3, 37), (3, 42), (17, 39), (7, 34), (1, 36), (1, 34), (11, 37), (42, 37)]
run model: 1.359307050704956
model num: 31 and [(15, 34), (15, 38), (38, 34), (38, 39), (39, 34), (30, 34), (30, 38), (30, 39), (3, 34), (3, 37), (28, 34), (28, 38), (0, 37), (0, 34), (13, 34), (13, 38), (13, 39), (1

In [ ]:
x = [i for i in range(len(full_rew_list))]
y = full_rew_list
plt.plot(x,y)

In [ ]:
x = [i for i in range(len(high_rew_list))]
y = [i for i,j in high_rew_list]
plt.plot(x,y)

In [ ]:
data = np.random.rand(dis_x, dis_y)
#0 = red, 1 = blue

for i in range(dis_x):
    for j in range(dis_y):
        fh=open(os.getcwd()+"\\Mixed_MAP\\model_"+str(i)+"_"+str(j)+".txt", "r")
        data[i][j] = fh.read()
        fh.close()
        MAP_gene_model = read_graph_MAP(i,j,num_inp,num_out)
        if(nx.is_empty(MAP_gene_model)):
            data[i][j] = -100000

#for i in range(dis_x):
#    print(data[i])
            
# create discrete colormap
cmap = mpl.cm.seismic
norm = mpl.colors.Normalize(vmin=-100000, vmax=100000)

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(data, cmap=cmap, norm=norm)

plt.show()

In [ ]:
#Fixes some weird tensorflow error
tf.compat.v1.disable_eager_execution()
env = gym.make('BipedalWalker-v3')
#get model number
#reward_list.sort(reverse=True)
#model_num = reward_list[0][1]
#print(model_num)
model_num = 29

#81, 56 at gen 24
#114, gen 91:  82,33
#1, gen 73: 98, 55

#x_val, y_val = 99,14
x_val, y_val = 0,2

#93,48

gene_model = read_graph_MAP(x_val,y_val,num_inp,num_out)

#get model from file
#gene_model = read_graph_NEAT(model_num,num_inp,num_out)

#turn gene model into tensorflow model
tf_outputs,calc_dict = get_tf_model(gene_model,num_inp,num_out)

#get inputs
observation = env.reset()
observation = np.append(observation,[1])

#run model
rewards = 0
game_step_sum = 0
avg_horizontal_speed = 0
with tf.Session() as sess:
    for game_steps in range(500):#num_game_steps):
        game_step_sum+=1
        #set up feed dictionary for model input
        feed_dict = {}
        for i in range(len(observation)): feed_dict[calc_dict[i]] = [observation[i]]
        for i in range(len(observation),num_inp): feed_dict[calc_dict[i]] = [1.0]

        fin_out = []
        actions = sess.run(tf_outputs,feed_dict=feed_dict)  #this is what costs the most time

        for i in range(num_out):
            fin_out.append(actions[i][0])
        #take action
        env.render()
        action = np.array(fin_out)
        observation, reward, done, info = env.step(action)
        avg_horizontal_speed += observation[2]
        rewards+=reward
        if(done):
            break
env.close()
print(rewards)
print(avg_horizontal_speed*game_step_sum)